In [1]:
import damask
import numpy as np
import pyvista as pv

pv.set_jupyter_backend('pythreejs')


In [2]:
# new material "material"
material = damask.ConfigMaterial()

In [3]:
# define isotrain Taylor 2 homogenization

#material['homogenization']['sx'] = damask.ConfigMaterial.load('isostrain_Taylor2.yaml') # does not work
material['homogenization']['sx'] = {'N_constituents':1, 'mechanical': {'type':'isostrain'}} # make it make sense

#material['homogenization']['sx'] = {'N_constituents':1}


In [4]:
#define materials, Material A being Al (plastic) and Material B being Ti (plastic)
material['phase']['A'] = damask.ConfigMaterial.load('phenopowerlaw_Al.yaml') # Load AL (plastic)
#material['phase']['B'] = damask.ConfigMaterial.load('phenopowerlaw_Ti.yaml') # Load Ti (plastic)

FileNotFoundError: [Errno 2] No such file or directory: 'phenopowerlaw_Al.yaml'

In [16]:
# define microstructure, 25 grains of Al and Ti - creating random microstructure
O_A = damask.Rotation.from_random(25)
O_B = damask.Rotation.from_random(25)

In [17]:
# add the materials
material = material.material_add(homogenization='sx', phase='A', O=O_A)
material = material.material_add(homogenization='sx', phase='B', O=O_B)

In [18]:
# export configurated material to material.yaml
material.save()

In [19]:
# setup polycrystal params
size = np.ones(3)*1e-5
cells = [64,64,64]
N_grains = 50
seeds = damask.seeds.from_random(size, N_grains, cells)

In [20]:
# defining crystal from triply-periodic minimal surface "Schwarz D"
#grid = damask.Grid.from_minimal_surface(cells, size, 'Schwarz D')
grid = damask.Grid.from_Voronoi_tessellation(cells, size, seeds)

In [21]:
# save grid as 50_64x64x64_pc.vti
grid.save(f'{N_grains}_{cells[0]}x{cells[1]}x{cells[2]}_pc_vor')

In [22]:
# visualize grid or don't 
org = pv.UniformGrid(dims=grid.cells+1,
                     spacing=grid.size/grid.cells,
                     origin=grid.origin)
org['material'] =   grid.material.flatten('F')

pl = pv.Plotter()
pl.set_background('white')
pl.add_mesh(org)
pl.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…